python3 -m pip install -U git+https://github.com/adefossez/demucs#egg=demucs

In [ ]:
import demucs.api
import soundfile as sf
import torchaudio
import torch
import scipy.io as sio
import tensorflow as tf
import scipy
import numpy as np
import csv
import pandas as pd
import time

# import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile

# import tensorflow_hub as hub
# import kagglehub
# path = kagglehub.model_download("google/yamnet/tensorFlow2/yamnet")
path = 'yamnet/tensorFlow2/yamnet/1'
model = tf.saved_model.load(path, tags=None, options=None)

In [ ]:
df_meta = pd.read_csv('STM_output/STM_metaData/metaData_fma_large.csv', index_col=0)
df_meta.head()

In [ ]:
# modified from here: https://www.tensorflow.org/hub/tutorials/yamnet
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
    """Returns list of class names corresponding to score vector."""
    class_names = []
    with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            class_names.append(row['display_name'])
    
    return class_names

class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)

# verify and convert a loaded audio is on the proper sample_rate (16K), otherwise it would affect the model's results.
def ensure_sample_rate(original_sample_rate, waveform, desired_sample_rate=16000):
    """Resample waveform if required."""
    if original_sample_rate != desired_sample_rate:
        desired_length = int(round(float(len(waveform)) / original_sample_rate * desired_sample_rate))
        waveform = scipy.signal.resample(waveform, desired_length)
    return desired_sample_rate, waveform

def estimate_voice(df_meta, n_row)
    # st = time.time()
    
    filename = df_meta['filepath'].iloc[n_row]
    frame_offset = df_meta['startPoint'].iloc[n_row]-1
    num_frame = df_meta['endPoint'].iloc[n_row]-frame_offset+1
    y_tensor, sr = torchaudio.load(filename, frame_offset=frame_offset, num_frames=num_frame)
    
    # if the audio is mono, duplicate the channel
    if y_tensor.shape[0]==1:
        y_tensor = torch.stack((y_tensor, y_tensor), dim=0)
    
    # demucs.api.Separator() by default will make sure the sampling rate is at 44.1 kHz
    separator = demucs.api.Separator()
    origin, separated = separator.separate_tensor(wav=y_tensor, sr=sr)
    
    # convert sampling rate and feed into YAMNet
    _, waveform = ensure_sample_rate(44100, np.array(separated['vocals'].mean(axis=0))) # convert to sr=16000
    scores, _, _ = model(np.array(separated['vocals'].mean(axis=0))) # use YAMNET to score the audio waveform
    scores_np = scores.numpy()
    infered_class = class_names[scores_np.mean(axis=0).argmax()]
    voice = infered_class in class_names[0:35] # whether the max likelihood category belongs to any labels of human voice

    # # get the execution time
    # et = time.time()
    # print('Execution time:', et - st, 'seconds')

    return voice

In [3]:
# not done yet...

import concurrent.futures

def process_data(data):
    # Process data here
    return data * 2

def parallel_processing(data_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks to the executor
        futures = [executor.submit(process_data, data) for data in data_list]
        
        # Gather results
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
        
    return results

if __name__ == "__main__":
    data_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    processed_data = parallel_processing(data_list)
    print(processed_data)


[4, 6, 8, 2, 20, 18, 16, 10, 14, 12]
